In [1]:
import numpy as np
import torch as th
import torch.nn as nn
import os
import json
import pandas as pd
import torchvision.transforms as T

from train.multi_hyperparameter import MultiHyperparameter
from custom_models.unet_original import UNet, UNet2
from evaluate.cross_evaluator import CrossTrainEvaluator
from train.unet_trainer import UnetTrainer
from datasets.semantic_dataset import SemanticDataset
from preprocessing.data_augment import DataAugmenter

C:\Users\silva\anaconda3\envs\TorchEnv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
params = {'padding_mode': ['zeros', 'reflect', 'replicate', 'circular'],
          'depth': 5,
          'start_layers': 64,
          'dim_multiplier': 2,
          'input_conv_kernel_size': 3,
          'out_classes': 2,
          'criterion': nn.CrossEntropyLoss(),
          'optimizer': 1}

param = {'padding_mode': 'zeros',
         'depth': 3,
         'start_layers': 32,
         'dim_multiplier': 2,
         'input_conv_kernel_size': 3,
         'out_classes': 2,
         'criterion': nn.CrossEntropyLoss(),
         'optimizer': 1,
         'augment_transform': T.Compose([T.RandomVerticalFlip()]),
         'num_augments': 10,
         'binarizer_lr': 0.05,
         'batch_size': 1}

unet_hyps = MultiHyperparameter(params)
unet = UNet.__new__(UNet)

models_to_evaluate = [(unet, unet_hyps)]

num_epochs = 1
models = []
trainers = []
folds = [0, 1, 2, 3]
device = th.device("cuda" if th.cuda.is_available() else "cpu")
cwd = os.getcwd()
ds1_path = os.path.join(cwd, '../data/cell_type_1')
ds2_path = os.path.join(cwd, '../data/cell_type_2')
result_path = os.path.join(cwd, '../results')

In [3]:
cte = CrossTrainEvaluator(unet, [ds1_path, ds2_path], device, result_path)
report = cte.evaluate_param(param, [0, 1, 2, 3], 20)

print(report)

{'padding_mode': 'zeros', 'depth': 3, 'start_layers': 32, 'dim_multiplier': 2, 'input_conv_kernel_size': 3, 'out_classes': 2, 'criterion': CrossEntropyLoss(), 'optimizer': 1, 'augment_transform': Compose(
    RandomVerticalFlip(p=0.5)
), 'num_augments': 10, 'binarizer_lr': 0.05, 'batch_size': 1}


KeyboardInterrupt: 

In [3]:
from postprocessing.binarization import Binarizer2Class

ds = SemanticDataset(ds1_path, [1])
test_unet = UNet(param).to(device)
binar = Binarizer2Class(device, 0.1, initial_threshold=0.5)

loader = th.utils.data.DataLoader(ds, batch_size=1, shuffle=True)
#
data = []
#
for item in loader:
    img, target = item
    target_store = target.clone().detach().to('cpu')
    target_store = target_store.to(th.int32)
    img = img.to(device)
    pred = test_unet.forward(img)
    pred_store = pred.clone().detach().to('cpu')
    data.append((target_store, pred_store))

loader = th.utils.data.DataLoader(data, batch_size=1, shuffle=True)
binar.train(loader)
print(binar.threshold)
binar.forward(th.rand((2, 2, 100, 100)))


0
0.0
0.0
0.0
0.0
0.0
0.0
0.6000000000000001
0.7000000000000001
0.8
0.9


tensor([[[[0, 0, 0,  ..., 0, 0, 1],
          [0, 0, 0,  ..., 0, 1, 0],
          [0, 0, 0,  ..., 1, 0, 0],
          ...,
          [1, 0, 0,  ..., 0, 0, 0],
          [1, 0, 0,  ..., 0, 1, 0],
          [0, 1, 1,  ..., 0, 0, 0]],

         [[0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0],
          ...,
          [0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 1, 0, 0]]],


        [[[0, 0, 1,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 1, 0],
          ...,
          [0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0]],

         [[0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0],
          ...,
          [0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0]]]], device='cuda:0')

AttributeError: 'builtin_function_or_method' object has no attribute 'train'

In [6]:
cte = CrossTrainEvaluator(unet, [ds1_path, ds2_path], device, result_path)
report = cte.evaluate_param(param, [0, 1, 2, 3], 1)

{'padding_mode': 'zeros', 'depth': 5, 'start_layers': 64, 'dim_multiplier': 2, 'input_conv_kernel_size': 3, 'out_classes': 2, 'criterion': CrossEntropyLoss(), 'optimizer': 1, 'augment_transform': Compose(
    RandomVerticalFlip(p=0.5)
), 'num_augments': 10}


KeyboardInterrupt: 

AttributeError: 'NoneType' object has no attribute 'forward'

In [ ]:
file = th.load(os.path.join(cwd, '../results/test.pt'))
print(file[0]['criterion'])